In [1]:
from __future__ import division
import imp
import uproot
import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import os
import ROOT
import time
import math
from array import array
import collections

inputdir = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted/'
outputdir = inputdir+'xsec_cross_check/' 
output_filedir = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted/xsec_cross_check/'
input_filedir = '/home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted/'
lib_function_dir = '/home/tmettler/Desktop/uBoone/do_plots/'

plot_systematic = 0

# helper functions
globale = imp.load_source('globale',lib_function_dir+'globale.py')
NuCC = imp.load_source('NuCC_function',lib_function_dir+'NuCC_function.py')
NuCC_w = imp.load_source('NuCCWeight_function',lib_function_dir+'NuCCWeight_function.py')

Welcome to JupyROOT 6.10/08


In [2]:
#!jupyter nbconvert --to script numuCCana_xsec_crosscheck.ipynb

In [3]:
# initialte ROOT default canvas
#ROOT.gROOT.SetBatch(ROOT.kFALSE)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetOptTitle(0)
c1 = ROOT.TCanvas("c1","c1",1600,1200)
c1.SetGrid(1)
c1.SetLeftMargin(0.14)
c1.SetRightMargin(0.18)
c1.SetBottomMargin(0.14)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


# CRTinBNB tagger, Overlay or MC

In [4]:
# Load input files
#inputdir = '/home/thomasm/numuCC/V33/10kevents/'
outputdir_png, outputdir_root, outputdir_pdf = NuCC.prepareOutput2(outputdir)

#filename_overlay = 'NuCCana_Overlay_V18_V4.root'
filename_overlay = 'NuCCana_overlay_V26_weight.root'
#filename_overlay = 'NuCCana_overlay_v25_weighted_all.root'
filename_data = 'NuCCana_data_V25.root'
filename_ext = 'NuCCana_ext_V25.root'
filename_dirt = 'NuCCana_dirt_V26_weight.root'
    
tree_name = 'numuCCAna'

In [5]:
#Open all the trees of the four files (data, ext, dirt, overlay)

data, ext, dirt, overlay = NuCC.openTrees(inputdir, filename_data, filename_ext, filename_dirt, filename_overlay, tree_name)
NuCC.printNumberOfEntries(data,ext,dirt,overlay)

pot_overlay = NuCC.getPOT(inputdir,filename_overlay,tree_name)
pot_dirt =  NuCC.getPOT(inputdir,filename_dirt,tree_name)
#V25 files
pot_data =    7.644e+18  # best with tor875
data_trigger = 1838700.0 #2220362.0 #1854495.0 #4743794 # 1987072.0 # E1DCNT_wcut
ext_trigger =  18997529.0  #2120135 #5685315 # EXT

print 'POT: '
print 'Data:\t\t', pot_data
print 'Ext:\t\t', 0
print 'Overlay:\t', pot_overlay
print 'Dirt:\t\t', pot_dirt
print ''
sample = [data,ext,overlay,dirt]
scale = {data:1.0,ext:1.0,overlay:1.0,dirt:1.0}
name = {data:'data',ext:'ext',overlay:'overlay',dirt:'dirt'}

scale[data], scale[ext], scale[dirt], scale[overlay] = NuCC.calculateScale(data_trigger, ext_trigger, pot_data, pot_dirt, pot_overlay)

scale[dirt] = scale[dirt]
scale[overlay] = scale[overlay]
print 'Scalefactors: '
print 'Data:\t\t', scale[data]
print 'Ext:\t\t', scale[ext]
print 'Overlay:\t', scale[overlay]
print 'Dirt:\t\t', scale[dirt]


Overlay: Number of Entries:	989745
Data: Number of Entries:	28005
Ext: Number of Entries:		222702
Dirt: Number of Entries:	40808

POT: 
Data:		7.644e+18
Ext:		0
Overlay:	1.24025430537e+21
Dirt:		1.25019694173e+20

Scalefactors: 
Data:		1.0
Ext:		0.0967862715198
Overlay:	0.00616325213863
Dirt:		0.0611423668132


Warning in <TClass::Init>: no dictionary for class ROOT::TIOFeatures is available


In [6]:
if 1:
    filename_overlay = filename_overlay+'out4.root'
    #filename_overlay = 'NuCCana_overlay_points_1kev.rootout2.root'
    filename_data = filename_data+'out4.root'
    filename_ext = filename_ext+'out4.root'
    filename_dirt = filename_dirt+'out4.root'

    tree_name = 't_out'

    data_out, ext_out, dirt_out, overlay_out = NuCC.openTreesOut(inputdir, filename_data, filename_ext, filename_dirt, filename_overlay, tree_name)
    NuCC.printNumberOfEntries(data_out,ext_out,dirt_out,overlay_out)

    sample_out = [data_out,ext_out,overlay_out,dirt_out]
    scale_out = {data_out:1.0,ext_out:1.0,overlay_out:1.0,dirt_out:1.0}
    name_out = {data_out:'data',ext_out:'ext',overlay_out:'overlay',dirt_out:'dirt'}

    scale_out[data_out], scale_out[ext_out], scale_out[dirt_out], scale_out[overlay_out] = NuCC.calculateScale(data_trigger, ext_trigger, pot_data, pot_dirt, pot_overlay)
    scale_out[dirt_out] = scale_out[dirt_out]
    scale_out[overlay_out] = scale_out[overlay_out]

Overlay: Number of Entries:	989714
Data: Number of Entries:	28004
Ext: Number of Entries:		222701
Dirt: Number of Entries:	40800



In [7]:
##### flux and number of tragets parameters###
flux = 1.16859e11/1.592e20 # flux per POT per cm2
print flux
flux = 7.3789785277e-10
print flux
roh_data = 1.3836 #data denisity g/cm3
roh_MC = 1.3954 # MC denisity g/cm3
mol = 39.95 # g for argon
N_A = 6.022140857e23 # molec/mol avogadro zahl
N_nucleons = 40.0
V_fid = ((254.8-10)-(-1.55+10))*((117.47-10)-(-115.53+10))*((1036.9-50)-(-0.1+10))
print 'Fiducial Volume: ', V_fid
##############################################

beam_flux = flux * pot_data
print 'Beam flux = {:.5e}'.format(beam_flux),' /cm2'
N_tot = roh_data*N_A*N_nucleons*V_fid/mol
print 'Number of target nuclei= {:.5e}'.format(N_tot),' /cm3'

7.34038944724e-10
7.3789785277e-10
Fiducial Volume:  49184671.35
Beam flux = 5.64049e+09  /cm2
Number of target nuclei= 4.10331e+31  /cm3


In [8]:
# Define signals

fidVol = '(Nu_Vx_sce>(-1.55+10) && Nu_Vx_sce<(254.8-10)) && (Nu_Vy_sce>(-115.53+10) && Nu_Vy_sce<(117.47-10)) &&(Nu_Vz_sce>(-0.1+10) && Nu_Vz_sce<(1036.9-50))'
MCfidVol = '(MCNu_Vx>(-1.55+10) && MCNu_Vx<(254.8-10)) && (MCNu_Vy>(-115.53+10) && MCNu_Vy<(117.47-10)) &&(MCNu_Vz>(-0.1+10) && MCNu_Vz<(1036.9-50))'
numu_signal = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14 && MCTrackPDG==13 && MCTrackPurity>0.5' # numu CC signal definition
numu_true = 'MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14' # numu CC signal definition
numu_nomu = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14 && MCTrackPDG!=13 && MCTrackPurity>0.5' # not an MC muon
numu_lowpur = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14 && MCTrackPurity<0.5' #low purity
numu_nc = 'fidVol && MCfidVol && MCNu_CCNC==1' # nutral current
numu_ov = 'fidVol && !MCfidVol' # out of fiducial
numu_other = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG!=14' # e.g anti nu or nue
#signal = 'MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14'
for x in sample:
    x.SetAlias('muon','(muon_candidate_key==track_key)')
    
num_fidVol = {}
for x in sample:
    x.SetAlias('fidVol',fidVol)
    x.SetAlias('MCfidVol',MCfidVol)
    x.SetAlias('numu_signal',numu_signal)
    x.SetAlias('numu_true',numu_true)
    x.SetAlias('numu_nomu',numu_nomu)
    x.SetAlias('numu_lowpur',numu_lowpur)
    x.SetAlias('numu_nc',numu_nc)
    x.SetAlias('numu_ov',numu_ov)
    x.SetAlias('numu_other',numu_other)
    num_fidVol[x] = x.GetEntries('fidVol && muon')*scale[x]
    
tot_num_fidVol = num_fidVol[ext]+num_fidVol[dirt]+num_fidVol[overlay]
overlay_signals = {'numu_signal','numu_nomu','numu_lowpur','numu_nc','numu_ov','numu_other'}

In [9]:
for x in sample_out:
    x.SetAlias('muon','(track_key == key_muon)')

num_fidVol = {}
for x in sample_out:
    x.SetAlias('fidVol',fidVol)
    x.SetAlias('MCfidVol',MCfidVol)
    x.SetAlias('numu_signal',numu_signal)
    x.SetAlias('numu_true',numu_true)
    x.SetAlias('numu_nomu',numu_nomu)
    x.SetAlias('numu_lowpur',numu_lowpur)
    x.SetAlias('numu_nc',numu_nc)
    x.SetAlias('numu_ov',numu_ov)
    x.SetAlias('numu_other',numu_other)

In [10]:
# define channels definitions

In [11]:
# Load the global variables for access of functions
NuCC.loadGlobal(data,ext,dirt,overlay,data_out,ext_out,dirt_out,overlay_out,scale,scale_out,tot_num_fidVol,overlay_signals,sample,sample_out, name,name_out, outputdir_png, outputdir_root,outputdir_pdf)
#NuCC.printGlobal()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [12]:
# initialte ROOT default canvas
ROOT.gStyle.SetOptStat(0)
c1 = ROOT.TCanvas("c1","c1",1600,1200)
c1.SetGrid(1)
c1.SetLeftMargin(0.14)
c1.SetRightMargin(0.18)
c1.SetBottomMargin(0.14)

In [13]:
'''track_start_border_x = '(TrackStart_x_sce <(-1.55+1) || TrackStart_x_sce > (254.8-1))'
track_end_border_x = '(TrackEnd_x_sce <(-1.55+1) || TrackEnd_x_sce > (254.8-1))'
track_start_border_y = '(TrackStart_y_sce <(-115.53+1) || TrackStart_y_sce > (117.47-1))'
track_end_border_y = '(TrackEnd_y_sce <(-115.53+1) || TrackEnd_y_sce > (117.47-1))'
track_start_border_z = '(TrackStart_z_sce <(0.1+1) || TrackStart_z_sce > (1036.9-1))'
track_end_border_z = '(TrackEnd_z_sce <(0.1+1) || TrackEnd_z_sce > (1039.9-1))'
'''
track_start_border_x = '(TrackStart_x_sce <(-1.55+5) || TrackStart_x_sce > (254.8-5))'
track_end_border_x = '(TrackEnd_x_sce <(-1.55+5) || TrackEnd_x_sce > (254.8-5))'
track_start_border_y = '(TrackStart_y_sce <(-115.53+5) || TrackStart_y_sce > (117.47-5))'
track_end_border_y = '(TrackEnd_y_sce <(-115.53+5) || TrackEnd_y_sce > (117.47-5))'
track_start_border_z = '(TrackStart_z_sce <(0.1+5) || TrackStart_z_sce > (1036.9-5))'
track_end_border_z = '(TrackEnd_z_sce <(0.1+5) || TrackEnd_z_sce > (1039.9-5))'

track_end_uncontained = '(' + track_end_border_x + ' || ' + track_end_border_y + ' || ' + track_end_border_z+ ')'


data.SetAlias("track_end_uncontained",track_end_uncontained)
ext.SetAlias("track_end_uncontained",track_end_uncontained)
overlay.SetAlias("track_end_uncontained",track_end_uncontained)
dirt.SetAlias("track_end_uncontained",track_end_uncontained)
data_out.SetAlias("track_end_uncontained",track_end_uncontained)
ext_out.SetAlias("track_end_uncontained",track_end_uncontained)
overlay_out.SetAlias("track_end_uncontained",track_end_uncontained)
dirt_out.SetAlias("track_end_uncontained",track_end_uncontained)

data.SetAlias("crt_cut","(abs(crtt0_time+(crt_trig_corr_med)/1000-4)<0.9 || crtt0_time==-1)")
ext.SetAlias("crt_cut","(abs(crtt0_time+(crt_trig_corr_med)/1000-3.57+3.195-4)<0.9 || crtt0_time==-1)")
overlay.SetAlias("crt_cut","(abs(crtt0_time-3.95)<0.9 || crtt0_time==-1)")
dirt.SetAlias("crt_cut","(abs(crtt0_time-3.95)<0.9 || crtt0_time==-1)")
data_out.SetAlias("crt_cut","(abs(crtt0_time+(crt_trig_corr_med)/1000-3.95)<0.9 || crtt0_time==-1)")
ext_out.SetAlias("crt_cut","(abs(crtt0_time+(crt_trig_corr_med)/1000-3.57+3.195-3.95)<0.9 || crtt0_time==-1)")
overlay_out.SetAlias("crt_cut","(abs(crtt0_time-3.95)<0.9 || crtt0_time==-1)")
dirt_out.SetAlias("crt_cut","(abs(crtt0_time-3.95)<0.9 || crtt0_time==-1)")

crt_tom_cut = 'nr_crthit_top==0 && crthit_vertex_zcut==0 && (track_end_uncontained==1 || nr_crthit_beam_tres==0) && crt_cut'

data.SetAlias("crt_tom_cut",crt_tom_cut)
ext.SetAlias("crt_tom_cut",crt_tom_cut)
overlay.SetAlias("crt_tom_cut",crt_tom_cut)
dirt.SetAlias("crt_tom_cut",crt_tom_cut)
data_out.SetAlias("crt_tom_cut",crt_tom_cut)
ext_out.SetAlias("crt_tom_cut",crt_tom_cut)
overlay_out.SetAlias("crt_tom_cut",crt_tom_cut)
dirt_out.SetAlias("crt_tom_cut",crt_tom_cut)

weight_name = 'EventWeight*TunedCentralValue_Genie'


In [14]:
# initialte ROOT default canvas
ROOT.gStyle.SetOptStat(0)
c1 = ROOT.TCanvas("c1","c1",1600,1200)
c1.SetGrid(1)
c1.SetLeftMargin(0.14)
c1.SetRightMargin(0.05)
c1.SetBottomMargin(0.14)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [15]:
def eff_channel_test(cut,variable,xstart,xend,xbins,axis_name,file_name,side='right'):
    #overlay_signals_func = ['other','DIS','COH','RES','MEC','QE']
    overlay_signals_func = ['QE','MEC','RES','DIS']
    weight_name = 'EventWeight*TunedCentralValue_Genie'
    globale.overlay_out.SetAlias('DIS','MCNu_Interaction==2')
    globale.overlay_out.SetAlias('COH','(MCNu_Interaction==3 || MCNu_Interaction==4)')
    globale.overlay_out.SetAlias('RES','MCNu_Interaction==1')
    globale.overlay_out.SetAlias('MEC','MCNu_Interaction==10')
    globale.overlay_out.SetAlias('QE','MCNu_Interaction==0')
    globale.overlay_out.SetAlias('other','!(DIS || COH || RES || MEC || QE)')
    
    h_overlay_func = {} # make an array of histograms for the 
    h_overlay_func_gen = {} # make an array of histograms for the 
    #tot_events = 0.0
    #channel_event = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    for x in overlay_signals_func:
        h_overlay_func[x] = ROOT.TH1F(x,'title',xbins,xstart,xend)
        h_overlay_func_gen[x] = ROOT.TH1F(x+'gen','title',xbins,xstart,xend)
    
    #hs = ROOT.THStack("hs",variable+';'+axis_name+'; Entries per bin');
    cut = cut +' && '
    cut_true = 'numu_true && '
    if variable == 'MCTrackPhi':
        cut_true = cut_true+'MCTrackPhi!=-1 && MCTrackPhi!=0 &&'
    if variable == 'MCTrackMomentum':
        cut_true = cut_true+'MCTrackMomentum!=-1 && MCTrackMomentum!=0 && '
        
    for i,x in enumerate(overlay_signals_func):
        histo = x
        globale.overlay_out.Draw(variable+'>>'+histo,weight_name+'*('+cut+x+')','')
        globale.overlay_out.Draw(variable+'>>'+histo+'gen',weight_name+'*('+cut_true+x+')','')
        h_overlay_func[x].Divide(h_overlay_func_gen[x])
        h_overlay_func[x].GetYaxis().SetTitleSize(0.05)
        h_overlay_func[x].GetYaxis().SetTitleOffset(0.0)
        h_overlay_func[x].GetYaxis().SetLabelSize(0.05)
        h_overlay_func[x].GetXaxis().SetTitleSize(0.05)
        h_overlay_func[x].GetXaxis().SetLabelSize(0.05)
        h_overlay_func[x].GetXaxis().SetTitleOffset(1)
        h_overlay_func[x].SetYTitle("Efficiency")
        h_overlay_func[x].SetXTitle(axis_name)
        h_overlay_func[x].SetLineWidth(4)
        #hs.Add(h_overlay_func[x])
    
    #h_overlay_func['other'].SetFillColor(ROOT.kGray+1)
    h_overlay_func['DIS'].SetFillColor(ROOT.kBlue+1)
    #h_overlay_func['COH'].SetFillColor(ROOT.kPink+10)
    h_overlay_func['RES'].SetFillColor(ROOT.kRed+1)
    h_overlay_func['MEC'].SetFillColor(ROOT.kOrange-3)
    h_overlay_func['QE'].SetFillColor(ROOT.kGreen+2)
    #h_overlay_func['other'].SetLineColor(ROOT.kGray+1)
    h_overlay_func['DIS'].SetLineColor(ROOT.kBlue+1)
    #h_overlay_func['COH'].SetLineColor(ROOT.kPink+10)
    h_overlay_func['RES'].SetLineColor(ROOT.kRed+1)
    h_overlay_func['MEC'].SetLineColor(ROOT.kOrange-3)
    h_overlay_func['QE'].SetLineColor(ROOT.kGreen+2)
    
    if side == 'left':
        legend = ROOT.TLegend(0.15,0.15,0.4,0.4) # LEGEND LEFT
    else:
        legend = ROOT.TLegend(0.7,0.15,0.95,0.4); #LEGEND RIGHT
        
    for i,x in enumerate(overlay_signals_func):
        #ov_name = overlay_signals_func[i]+': {0:0.2f}%'.format(channel_event[i]*100.0/(tot_events+1e-10))
        legend.AddEntry(h_overlay_func[x],overlay_signals_func[i],"f");
        
    c1 = ROOT.TCanvas("c1","c1",1600,1200)
    c1.SetGrid(1)
    c1.SetLeftMargin(0.14)
    c1.SetRightMargin(0.05)
    c1.SetBottomMargin(0.14)
    
    prelim = ROOT.TLatex(0.9,0.93, "MicroBooNE Simulation Preliminary");
    prelim.SetTextFont(62);
    prelim.SetTextColor(ROOT.kGray+2);
    prelim.SetNDC();
    prelim.SetTextSize(1/25.);
    prelim.SetTextAlign(32);

    #hs.SetYTitle("Entries per bin")
    h_overlay_func['DIS'].SetMinimum(0)
    h_overlay_func['DIS'].SetMaximum(1)
    
    #hs.SetLineColor(ROOT.kBlack)
    #hs.SetLineWidth(4)
    ROOT.gStyle.SetEndErrorSize(5)

    #h_overlay_func['other'].Draw('E1')
    h_overlay_func['DIS'].Draw('E1')
    #h_overlay_func['COH'].Draw('E1 same')
    h_overlay_func['RES'].Draw('E1 same')
    h_overlay_func['MEC'].Draw('E1 same')
    h_overlay_func['QE'].Draw('E1 same')
    legend.Draw()
    
    prelim.Draw()
    
    c1.Draw()
    c1.SaveAs(globale.outputdir_png + "h_channels_eff_"+file_name+".png")
    c1.SaveAs(globale.outputdir_root + "h_channels_eff_"+file_name+".root")
    c1.SaveAs(globale.outputdir_pdf+ "h_channels_eff_"+file_name + ".pdf")
    
    return 

In [16]:
cut = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && numu_signal'

In [17]:
#numu_signal = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14 && MCTrackPDG==13 && MCTrackPurity>0.5' # numu CC signal definition

cut = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && numu_signal'
name = 'all_cut'
#NuCC_w.make_channel_histo(cut,'MCNu_Energy',0,2.5,20,'True neutrino energy [GeV]','true_energy_'+name,'right')
#eff_channel_test(cut,'cos(MCNu_leptonTheta)',-1,1,20,'True cos(theta)','true_cos(theta)_'+name,'right')
#NuCC_w.make_channel_histo(cut,'MCle_Energy',0,2.5,20,'True muon momentum [GeV]','true_momentum_'+name,'right')
#NuCC_w.make_channel_histo(cut,'MCle_Phi',-3.15,3.15,20,'True muon phi','true_phi_'+name,'right')
#NuCC_w.make_channel_histo(cut + ' && 1','NuTracks',0,10,10,'Number of Tracks','NuTracks'+name,'right')

In [18]:
cut = 'numu_true'
name = 'generated'
#NuCC_w.make_channel_histo(cut,'MCNu_Energy',0,2.5,20,'Truth neutrino energy [GeV]','true_energy_'+name,'right')
#NuCC_w.make_channel_histo(cut,'cos(MCNu_leptonTheta)',-1,1,20,'True cos(theta)','true_cos(theta)_'+name,'left')
#NuCC_w.make_channel_histo(cut,'MCle_Energy',0,2.5,20,'Truth muon momentum [GeV]','true_momentum_'+name,'right')
#NuCC_w.make_channel_histo(cut,'MCle_Phi',-3.15,3.15,20,'Truth muon phi','true_phi_'+name,'right')

In [19]:
cut = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8\
                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1 && numu_signal'

#NuCC_w.eff_channel_histo(cut,'MCNu_Energy',0.2,0.4,200,'Truth neutrino energy [GeV]','true_energy_fine','right')
#NuCC_w.eff_channel_histo(cut,'MCNu_Energy',0.2,0.4,20,'Truth neutrino energy [GeV]','true_energy_rough','right')
NuCC_w.eff_channel_histo(cut,'MCNu_Energy',0,2.5,20,'Generated neutrino energy [GeV]','true_energy','right')

#NuCC_w.eff_channel_histo(cut,'MCle_Energy',0.1,0.3,200,'Truth muon momentum [GeV]','true_momentum_fine','right')
#NuCC_w.eff_channel_histo(cut,'MCle_Energy',0.1,0.3,20,'Truth muon momentum [GeV]','true_momentum_rough','right')
#NuCC_w.eff_channel_histo(cut,'MCle_Energy',0,2.5,20,'Truth muon momentum [GeV]','true_momentum','right')

NuCC_w.eff_channel_histo(cut,'cos(MCNu_leptonTheta)',-1,1,20,'Generated cos(theta)','true_cos(theta)','left')
NuCC_w.eff_channel_histo(cut,'MCle_Energy',0,1,10,'Generated muon momentum [GeV]','true_momentum','right')
NuCC_w.eff_channel_histo(cut,'MCle_Phi',-3.15,3.15,20,'Generated muon phi','true_phi','left')

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file /home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted/xsec_cross_check/png/h_channels_eff_true_energy.png has been created
Info in <TCanvas::SaveAs>: ROOT file /home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted/xsec_cross_check/root/h_channels_eff_true_energy.root has been created
Info in <TCanvas::Print>: pdf file /home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted/xsec_cross_check/pdf/h_channels_eff_true_energy.pdf has been created
Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: png file /home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted/xsec_cross_check/png/h_channels_eff_true_cos(theta).png has been created
Info in <TCanvas::SaveAs>: ROOT file /home/tmettler/Desktop/v08_00_00_33/V08_00_00_35/weighted/xsec_cross_check/root/h_channels_eff_

In [20]:
asawdf

NameError: name 'asawdf' is not defined

In [ ]:
cut = 'fidVol && muon'
name = '_particle_01'
side_right = 'right'
side_left = 'left'

#NuCC_w.make_stacked_histo_particle(cut,'Nu_Vx_sce','EventWeight','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+name,side_left)
#NuCC_w.make_stacked_histo_particle(cut,'Nu_Vy_sce','EventWeight','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+name,side_left)
#NuCC_w.make_stacked_histo_particle(cut,'Nu_Vz_sce','EventWeight','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+name,side_left)
#NuCC_w.make_stacked_histo_particle(cut,'TrackMomMCS_mom','EventWeight','Reco. muon track momentum (MCS) [GeV]',0,1.5,20,'TrackMom'+name,side_right)
#NuCC_w.make_stacked_histo_particle(cut,'cos(TrackTheta)','EventWeight','Cos(theta)',-1,1,20,'cosTheta'+name,side_left)
#NuCC_w.make_stacked_histo_particle(cut,'TrackPhi','EventWeight','Reco. muon phi',-3.15,3.15,20,'phi'+name,side_left)

#NuCC_w.make_stacked_histo_particle(cut,'TrackLength','EventWeight','muon track length [cm]',0,600,50,'TrackLength'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut,'TrackPID_chiproton','EventWeight','Track PID Chi2 proton',0,350,20,'PIDproton'+name,side_left)
#NuCC_w.make_stacked_histo_particle(cut,'TrackPID_chimuon','EventWeight','Track PID Chi2 muon',0,60,20,'PIDmuon'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut,'TrackPID_chipion','EventWeight','Track PID Chi2 pion',0,60,20,'PIDpion'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut+'&& TrackPID_chimuon!=-999','TrackPID_chimuon/TrackPID_chiproton','EventWeight','Track PID Chi2 muon/proton ration',0,0.25,20,'PIDmuprot_ration'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut,'TrackPID_chimuon/TrackPID_chipion','EventWeight','Track PID Chi2 muon/pion ration',0.4,1.2,20,'PIDmupion_ration'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut,'nr_crthit_top','EventWeight','Number of CRT hits in top',0,4,4,'CRTTophit'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut,'crthit_vertex_zcut','EventWeight','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut,'TrackScore','EventWeight','TrackScore',0.5,1,20,'TrackScore'+name,side_left)
#NuCC_w.make_stacked_histo_particle(cut,'NuScore','EventWeight','Topological score',0,1,20,'NuScore'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut + ' && 1','NuTracks','EventWeight','Number of Tracks',0,10,10,'NuTracks'+name,side_right)
#NuCC_w.make_stacked_histo_particle(cut + ' && 1','NumPfp','EventWeight','Number of Pfparticle',0,10,10,'NumPfp'+name,side_right)

In [ ]:
cut = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1'
name = '_07'
side_right = 'right'
side_left = 'left'

#NuCC_w.make_stacked_histo_particle(cut,'Nu_Vx_sce','EventWeight','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+name,side_left)
#NuCC_w.make_stacked_histo_particle(cut,'Nu_Vy_sce','EventWeight','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+name,side_left)
#NuCC_w.make_stacked_histo_particle(cut,'Nu_Vz_sce','EventWeight','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+name,side_left)
#NuCC_w.make_stacked_histo_particle(cut,'TrackMomMCS_mom','EventWeight','Reco. muon track momentum (MCS) [GeV]',0,1.5,20,'TrackMom'+name,side_right)
#NuCC_w.make_stacked_histo_particle(cut,'cos(TrackTheta)','EventWeight','Cos(theta)',-1,1,20,'cosTheta'+name,side_left)
#NuCC_w.make_stacked_histo_particle(cut,'TrackPhi','EventWeight','Reco. muon phi',-3.15,3.15,20,'phi'+name,side_left)

#NuCC_w.make_stacked_histo_particle(cut,'TrackLength','EventWeight','muon track length [cm]',0,600,50,'TrackLength'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut,'TrackPID_chiproton','EventWeight','Track PID Chi2 proton',0,350,20,'PIDproton'+name,side_left)
#NuCC_w.make_stacked_histo_particle(cut,'TrackPID_chimuon','EventWeight','Track PID Chi2 muon',0,60,20,'PIDmuon'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut,'TrackPID_chipion','EventWeight','Track PID Chi2 pion',0,60,20,'PIDpion'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut+'&& TrackPID_chimuon!=-999','TrackPID_chimuon/TrackPID_chiproton','EventWeight','Track PID Chi2 muon/proton ration',0,0.25,20,'PIDmuprot_ration'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut,'TrackPID_chimuon/TrackPID_chipion','EventWeight','Track PID Chi2 muon/pion ration',0.4,1.2,20,'PIDmupion_ration'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut,'nr_crthit_top','EventWeight','Number of CRT hits in top',0,4,4,'CRTTophit'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut,'crthit_vertex_zcut','EventWeight','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut,'TrackScore','EventWeight','TrackScore',0.5,1,20,'TrackScore'+name,side_left)
#NuCC_w.make_stacked_histo_particle(cut,'NuScore','EventWeight','Topological score',0,1,20,'NuScore'+name,'right')
#NuCC_w.make_stacked_histo_particle(cut + ' && 1','NuTracks','EventWeight','Number of Tracks',0,10,10,'NuTracks'+name,side_right)
#NuCC_w.make_stacked_histo_particle(cut + ' && 1','NumPfp','EventWeight','Number of Pfparticle',0,10,10,'NumPfp'+name,side_right)

In [ ]:
cut = 'fidVol && muon'
name = '_01_PeLLE'
side_right = 'right'
side_left = 'left'
print 'preselection'
#NuCC_w.make_stacked_histo_weightV2(cut,'Nu_Vx_sce','EventWeight','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+name,side_left)
#NuCC_w.make_stacked_histo_weightV2(cut,'Nu_Vy_sce','EventWeight','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+name,side_left)
#NuCC_w.make_stacked_histo_weightV2(cut,'Nu_Vz_sce','EventWeight','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+name,side_left)
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackMomMCS_mom','EventWeight','Reco. muon track momentum (MCS) [GeV]',0.15,1.15,25,'TrackMom'+name,side_right)
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackMomRange_mu','EventWeight','Reco. muon track momentum (range) [GeV]',0.15,1.15,25,'TrackMomRange'+name,side_right)

#NuCC_w.make_stacked_histo_weightV2(cut,'cos(TrackTheta)','EventWeight','Cos(theta)',-1,1,28,'cosTheta'+name,side_left)
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackPhi','EventWeight','Reco. muon phi',-3.15,3.15,20,'phi'+name,side_left)

#NuCC_w.make_stacked_histo_weightV2(cut,'TrackLength','EventWeight','muon track length [cm]',0,500,25,'TrackLength'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackPID_chiproton','EventWeight','Track PID Chi2 proton',0,350,20,'PIDproton'+name,side_left)
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackPID_chimuon','EventWeight','Track PID Chi2 muon',0,60,20,'PIDmuon'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackPID_chipion','EventWeight','Track PID Chi2 pion',0,60,20,'PIDpion'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut+'&& TrackPID_chimuon!=-999','TrackPID_chimuon/TrackPID_chiproton','EventWeight','Track PID Chi2 muon/proton ration',0,0.25,20,'PIDmuprot_ration'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackPID_chimuon/TrackPID_chipion','EventWeight','Track PID Chi2 muon/pion ration',0.4,1.2,20,'PIDmupion_ration'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut,'nr_crthit_top','EventWeight','Number of CRT hits in top',0,4,4,'CRTTophit'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut,'crthit_vertex_zcut','EventWeight','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackScore','EventWeight','TrackScore',0,1,25,'TrackScore'+name,side_left)
#NuCC_w.make_stacked_histo_weightV2(cut,'NuScore','EventWeight','Topological score',0,1,25,'NuScore'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut + ' && 1','NuTracks','EventWeight','Number of Tracks',0,10,10,'NuTracks'+name,side_right)
#NuCC_w.make_stacked_histo_weightV2(cut + ' && 1','NumPfp','EventWeight','Number of Pfparticle',0,10,10,'NumPfp'+name,side_right)

In [ ]:
cut = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1'
name = '_07_PeLLE'
side_right = 'right'
side_left = 'left'
print 'preselection'
#NuCC_w.make_stacked_histo_weightV2(cut,'Nu_Vx_sce','EventWeight','Reco. Nu vertex X position [cm]',-10,270,20,'NuVx_sce'+name,side_left)
#NuCC_w.make_stacked_histo_weightV2(cut,'Nu_Vy_sce','EventWeight','Reco. Nu vertex Y position [cm]',-120,120,20,'NuVy_sce'+name,side_left)
#NuCC_w.make_stacked_histo_weightV2(cut,'Nu_Vz_sce','EventWeight','Reco. Nu vertex Z position [cm]',-50,1050,20,'NuVz_sce'+name,side_left)
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackMomMCS_mom','EventWeight','Reco. muon track momentum (MCS) [GeV]',0.15,1.15,10,'TrackMom'+name,side_right)
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackMomRange_mu','EventWeight','Reco. muon track momentum (range) [GeV]',0.15,1.15,10,'TrackMomRange'+name,side_right)
NuCC_w.make_stacked_histo_weightV2(cut+' && !track_end_uncontained','TrackMomRange_mu','EventWeight','Reco. muon track momentum (range) [GeV]',0.15,1.15,10,'TrackMomRange'+name+'_contained',side_right)

#NuCC_w.make_stacked_histo_weightV2(cut,'cos(TrackTheta)','EventWeight','Cos(theta)',-1,1,28,'cosTheta'+name,side_left)
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackPhi','EventWeight','Reco. muon phi',-3.15,3.15,20,'phi'+name,side_left)

#NuCC_w.make_stacked_histo_weightV2(cut,'TrackLength','EventWeight','muon track length [cm]',0,500,25,'TrackLength'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackPID_chiproton','EventWeight','Track PID Chi2 proton',0,350,20,'PIDproton'+name,side_left)
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackPID_chimuon','EventWeight','Track PID Chi2 muon',0,60,20,'PIDmuon'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackPID_chipion','EventWeight','Track PID Chi2 pion',0,60,20,'PIDpion'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut+'&& TrackPID_chimuon!=-999','TrackPID_chimuon/TrackPID_chiproton','EventWeight','Track PID Chi2 muon/proton ration',0,0.25,20,'PIDmuprot_ration'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackPID_chimuon/TrackPID_chipion','EventWeight','Track PID Chi2 muon/pion ration',0.4,1.2,20,'PIDmupion_ration'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut,'nr_crthit_top','EventWeight','Number of CRT hits in top',0,4,4,'CRTTophit'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut,'crthit_vertex_zcut','EventWeight','Number of CRT hits upstream of vertex',0,4,4,'CRTVertexcut'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut,'TrackScore','EventWeight','TrackScore',0,1,25,'TrackScore'+name,side_left)
#NuCC_w.make_stacked_histo_weightV2(cut,'NuScore','EventWeight','Topological score',0,1,25,'NuScore'+name,'right')
#NuCC_w.make_stacked_histo_weightV2(cut + ' && 1','NuTracks','EventWeight','Number of Tracks',0,10,10,'NuTracks'+name,side_right)
#NuCC_w.make_stacked_histo_weightV2(cut + ' && 1','NumPfp','EventWeight','Number of Pfparticle',0,10,10,'NumPfp'+name,side_right)

In [ ]:

cut = 'fidVol && muon && TrackLength>8 && crt_tom_cut && TrackScore>0.8                && TrackLength>20 && TrackPID_chiproton>78 && NuScore>0.1'
cut = cut + '&& abs(TrackMomRange_mu-TrackMomMCS_mom)<0.2'
h_migration = ROOT.TH2F("h_migration",'Truth vs. Reco',200,0,2.5,200,0,2.5)
globale.overlay_out.Draw('TrackMomRange_mu:TrackMomMCS_mom'+'>>h_migration',weight_name+'*('+cut+'&& numu_true)','')
h_migration.SetXTitle("reco Muon momentum MCS [GeV]")
h_migration.SetYTitle("reco Muon momentum range [GeV]")
h_migration.GetYaxis().SetTitleSize(0.05)
h_migration.GetYaxis().SetTitleOffset(0.0)
h_migration.GetYaxis().SetLabelSize(0.05)
h_migration.GetXaxis().SetTitleSize(0.05)
h_migration.GetXaxis().SetLabelSize(0.05)
h_migration.GetXaxis().SetTitleOffset(1)
h_migration.SetLineColor(ROOT.kBlack)
h_migration.SetLineWidth(4)
ROOT.gStyle.SetEndErrorSize(5)
c1.SetRightMargin(0.1)

h_migration.Draw("colz")
h_prof = h_migration.ProfileX()
h_prof.SetLineColor(ROOT.kRed)
h_prof.Draw('same')

#prelim = draw_sim()
#prelim.Draw()
#h_migration.Draw("same text")
c1.Draw()
c1.SaveAs(globale.outputdir_root + "h2_range_MCS.root")
c1.SaveAs(globale.outputdir_png + "h2_range_MCS.png")
h_migration.Write("h2_range_MCS_fine")